In [10]:
# Import the required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import MNIST
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os


# Fix the randomness
seed = 123456
torch.manual_seed(seed)

In [11]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
HP = {
    "lr": [1e-2,1e-3,1e-5],
    "neurons": [1,2,3],
    "hidden_layers": [[200, 100, 50], [200, 100], [200]],
    "activation_funcs": [nn.ReLU(),nn.Sigmoid()],
    "batch_size":128,
    "num_epoch":30,
    "patience":5
}

In [13]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as T

train_transform = T.Compose ([
T.ToTensor(),
T.Grayscale(),
T.Normalize(mean =(0.5,), std=(0.5,))
])
val_transform = test_transform = T.Compose([
T.ToTensor(),
T.Grayscale(),
T.Normalize(mean =(0.5,) , std=(0.5,))
])

train_set = CIFAR10 ( root ="CIFAR10", train =True ,transform = train_transform , download = True )
test_set = CIFAR10 ( root ="CIFAR10", train =False ,transform = test_transform , download = True )
train_set_length = int(0.8 * len(train_set))
val_set_length = len(train_set) - train_set_length
train_set, val_set = random_split(train_set, [train_set_length, val_set_length])


train_loader = DataLoader(train_set, batch_size=HP["batch_size"], shuffle=True,num_workers=6)
test_loader = DataLoader(test_set, batch_size=HP["batch_size"],num_workers=6)
val_loader = DataLoader(val_set, batch_size=HP["batch_size"],num_workers=6)


Files already downloaded and verified
Files already downloaded and verified


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
# Define the ANN
class MyModel(nn.Module):

    def __init__(self, output_shape, hidden_layers,activation):
        super().__init__()
        
        self.activation = activation

        layers = [32*32*1] + hidden_layers + [output_shape]
        self.mods = nn.ModuleList()

        for i in range(len(layers)-1):
            self.mods.append(nn.Linear(layers[i], layers[i+1]))

    def forward(self, x):
        x = torch.flatten(x, 1)
        for i in range(len(self.mods)-1):
          x = self.activation(self.mods[i](x))
        x = self.mods[-1](x)
        return x

In [16]:
def fit(m,i,j,k):
    patience = HP["patience"]
    prev_val_loss = np.inf
    best_model_at_epoch = 0
    for epoch in range(HP["num_epoch"]):
        # Training
        model.train()
        accum_train_loss = 0
        for n, (imgs, labels) in tqdm(enumerate(train_loader, start=1), total=len(train_loader), desc=f"Epoch: {epoch}"):
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)
            loss = loss_function(output, labels)

            # accumlate the loss
            accum_train_loss += loss.item()

            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        accum_val_loss = 0
        with torch.no_grad():
            for x, (imgs, labels) in enumerate(val_loader, start=1):
                imgs, labels = imgs.to(device), labels.to(device)
                output = model(imgs)
                accum_val_loss += loss_function(output, labels).item()
        
        # print statistics of the epoch
        train_loss = accum_train_loss / n
        val_loss = accum_val_loss / x
        print(f'Train Loss = {train_loss:.4f}\tVal Loss = {val_loss:.4f}\tPatience: {patience}')

        if(patience == 0):
            print("broken")
            break
        elif(val_loss > prev_val_loss):
            print("mmh")
            patience = patience - 1
        else:
            best_model_at_epoch = epoch
            prev_val_loss = val_loss
            patience = HP["patience"]

    return [HP["neurons"][m],HP["hidden_layers"][i],HP["lr"][j],HP["activation_funcs"][k],best_model_at_epoch, train_loss,val_loss]

In [17]:
def test(model):
    # Compute Test Accuracy
    model.eval()
    with torch.no_grad():
        correct = total = 0
        for images, labels in tqdm(test_loader,total=len(test_loader)):
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            
            _, predicted_labels = torch.max(output, 1)
            correct += (predicted_labels == labels).sum()
            total += labels.size(0)

    print(f'Test Accuracy = {100 * correct/total :.3f}%')
    return 100 * correct/total

In [18]:
results = []
c= 1
for m in range(len(HP["neurons"])):
    for i in range(len(HP["hidden_layers"])):
        for j in range(len(HP["lr"])):
            for k in range(len(HP["activation_funcs"])):
                print(f"----- Model: {c} -----")
                model = MyModel(10, hidden_layers=((np.array(HP["hidden_layers"][i])*HP["neurons"][m]).tolist()),activation=HP["activation_funcs"][k]).to(device)
                loss_function = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=HP["lr"][j])
                a = fit(m,i,j,k)
                a.append(test(model)[0])
                print(a)
                results.append(a)
                c=c+1
print(c)        
                

----- Model: 1 -----


Epoch: 0: 100%|██████████| 313/313 [00:06<00:00, 45.89it/s] 


Train Loss = 2.0018	Val Loss = 1.9465	Patience: 5


Epoch: 1: 100%|██████████| 313/313 [00:02<00:00, 104.70it/s]


Train Loss = 1.8974	Val Loss = 1.8985	Patience: 5


Epoch: 2: 100%|██████████| 313/313 [00:03<00:00, 96.13it/s] 


In [ ]:
epoch = 0
val_loss = [6,5,4,3,2,1,4,3,2,1,3,4,5,9,0]
patience = 5
i=0
prev_val_loss = float("inf")
while(True):
    print("i",i)
    if(patience == 0):
        print("broken")
        break
    elif(val_loss[i] > prev_val_loss):
        print("mmh")
        patience = patience - 1
    else:
        best_model_at_epoch = epoch
        prev_val_loss = val_loss[i]
        patience = 5
    print(val_loss[i])
    print("best",best_model_at_epoch)
    epoch = epoch +1
    i = i+1

i 0
6
best 0
i 1
5
best 1
i 2
4
best 2
i 3
3
best 3
i 4
2
best 4
i 5
1
best 5
i 6
mmh
4
best 5
i 7
mmh
3
best 5
i 8
mmh
2
best 5
i 9
1
best 9
i 10
mmh
3
best 9
i 11
mmh
4
best 9
i 12
mmh
5
best 9
i 13
mmh
9
best 9
i 14
0
best 14
i 15


IndexError: list index out of range

In [3]:
results = [[1,123,3,4,4,5],[1,1,1,1,1,1]]
fields= ["neurons","hidden_layers","lr","activation_funcs","best_epoch","train_loss","val_loss","test_accuracy"]
import csv
with open('RESULTS', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(results)


In [ ]:
import csv
    return [HP["neurons"][m],HP["hidden_layers"][i],HP["lr"][j],HP["activation_funcs"][k],best_model_at_epoch, train_loss,val_loss]

# field names
fields = ['Name', 'Branch', 'Year', 'CGPA']
	
# data rows of csv file
rows = [ ['Nikhil', 'COE', '2', '9.0'],
		['Sanchit', 'COE', '2', '9.1'],
		['Aditya', 'IT', '2', '9.3'],
		['Sagar', 'SE', '1', '9.5'],
		['Prateek', 'MCE', '3', '7.8'],
		['Sahil', 'EP', '2', '9.1']]

with open('GFG', 'w') as f:
	
	# using csv.writer method from CSV package
	write = csv.writer(f)
	
	write.writerow(fields)
	write.writerows(rows)
